In [3]:
# ================================================================
# Conv-4 on CIFAR-10 + Filter Fault Injection Benchmark (with JSON Logging)
# ================================================================
!pip install --quiet torch torchvision tqdm

import math, random, os, pathlib, torch, torch.nn as nn, torch.optim as optim
import torchvision, torchvision.transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

# ----------------------------- CONFIG ---------------------------
NUM_EPOCHS        = 3
BATCH_SIZE        = 128
LR                = 0.1
DEVICE            = 'cuda' if torch.cuda.is_available() else 'cpu'

FAULT_PERCENT     = 10          # % of *all* filters to random-re-draw
RANDOM_SEED       = 42
PRETRAINED_PATH   = ''          # leave empty to train from scratch
SAVE_CHECKPOINT_TO= '/content/conv4_cifar10.pt'

torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# -------------------------- DATASET -----------------------------
transform_train = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test  = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
test_loader  = DataLoader(testset,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# --------------------------- MODEL ------------------------------
class ConvBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
    def forward(self, x): return self.block(x)

class Conv4(nn.Module):
    def __init__(self, num_classes=10, in_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            ConvBlock(in_channels),
            ConvBlock(64),
            ConvBlock(64),
            ConvBlock(64),
        )
        self.classifier = nn.Linear(64 * 2 * 2, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)    # flatten
        return self.classifier(x)

model = Conv4().to(DEVICE)

# ---------------------- TRAIN / LOAD ----------------------------
def accuracy(net, loader):
    net.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            preds = net(images).argmax(1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)
    return 100. * correct / total

if PRETRAINED_PATH and pathlib.Path(PRETRAINED_PATH).exists():
    model.load_state_dict(torch.load(PRETRAINED_PATH, map_location=DEVICE))
    print(f'Loaded pretrained weights from {PRETRAINED_PATH}')
else:
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                    milestones=[NUM_EPOCHS//2, int(NUM_EPOCHS*0.75)], gamma=0.1)

    for epoch in range(NUM_EPOCHS):
        model.train()
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{NUM_EPOCHS}')
        for images, labels in pbar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(images), labels)
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'loss': f'{loss.item():.3f}'})
        scheduler.step()

    torch.save(model.state_dict(), SAVE_CHECKPOINT_TO)
    print(f'Saved checkpoint to {SAVE_CHECKPOINT_TO}')

base_acc = accuracy(model, test_loader)
print(f'\nBaseline accuracy: {base_acc:5.2f} %')

# ------------------ FILTER FAULT INJECTION ----------------------
def redraw_filters(net, percent: float, sigma: float = 0.05, save_json_path='filter_faults.json'):
    """
    Randomly re-draw `percent` % of filters from N(0, sigma²), evenly across Conv layers.
    Save changed filters and their indices in a JSON file.
    """
    conv_weights = [p for p in net.parameters() if p.ndim == 4]  # 4D conv filters
    total_filters = sum(p.size(0) for p in conv_weights)
    k = math.floor(total_filters * percent / 100 + 1e-6)

    # Equal distribution across layers
    per_layer = [math.floor(k / len(conv_weights))] * len(conv_weights)
    for i in range(k - sum(per_layer)):
        per_layer[i] += 1

    torch.manual_seed(RANDOM_SEED)
    changed_filters = []

    for layer_idx, (p, n_fault) in enumerate(zip(conv_weights, per_layer)):
        if n_fault == 0:
            continue
        idx_list = random.sample(range(p.size(0)), n_fault)
        for idx in idx_list:
            original = p[idx].detach().cpu().numpy().tolist()
            noise = torch.randn_like(p[idx]) * sigma
            p.data[idx] = noise
            modified = p[idx].detach().cpu().numpy().tolist()
            changed_filters.append({
                "layer": layer_idx,
                "filter_index": idx,
                "original_filter": original,
                "modified_filter": modified
            })

    # Save to JSON
    with open(save_json_path, 'w') as f:
        json.dump(changed_filters, f, indent=2)

    print(f'\nSaved filter fault info to: {save_json_path}')
    print(f'Preview of first few modified filters:')
    for entry in changed_filters[:3]:
        print(f"\nLayer {entry['layer']} | Filter {entry['filter_index']}")
        print("Original filter weights:", entry['original_filter'])
        print("Modified filter weights:", entry['modified_filter'])

print(f'\nInjecting random-re-draw fault into {FAULT_PERCENT} % of all filters …')
redraw_filters(model, FAULT_PERCENT, save_json_path='filter_faults.json')
faulty_acc = accuracy(model, test_loader)
print(f'Post-fault accuracy: {faulty_acc:5.2f} %')

print(f'\nAccuracy drop: {base_acc - faulty_acc:5.2f} percentage points')


Epoch 3/3: 100%|██████████| 391/391 [00:21<00:00, 18.16it/s, loss=0.882]

Saved checkpoint to /content/conv4_cifar10.pt



Baseline accuracy: 65.90 %

Injecting random-re-draw fault into 10 % of all filters …

Saved filter fault info to: filter_faults.json
Preview of first few modified filters:

Layer 0 | Filter 14
Original filter weights: [[[-0.025583768263459206, -0.22822798788547516, 0.13870789110660553], [-0.19518321752548218, 0.03616151586174965, 0.17975522577762604], [0.004639085382223129, 0.11092764139175415, -0.2676488757133484]], [[-0.13350507616996765, 0.06978676468133926, 0.07439516484737396], [-0.046199340373277664, -0.028306545689702034, 0.21517762541770935], [0.15136569738388062, 0.0892552062869072, -0.1464972048997879]], [[-0.19809497892856598, 0.05169738829135895, 0.24041733145713806], [-0.08267105370759964, 0.03758010268211365, 0.27104777097702026], [0.11142636835575104, 0.09242378920316696, -0.06519197672605515]]]
Modified filter weights: [[[0.009700939990580082, 0.10806868225336075, -0.008602511137723923], [0.04245300590991974, -0.09621994942426682, 0.03264927491545677], [-0.03247204050